In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split 
from sklearn.preprocessing import StandardScaler
from sklearn.feature_selection import RFE
from sklearn.svm import SVC
from sklearn.metrics import confusion_matrix, accuracy_score, classification_report

# Define functions

def rfeFeature(indep_X, dep_Y, n):
    rfelist = []
    selected_columns = []
    
    svc_model = SVC(kernel='linear', random_state=0)
    
    log_rfe = RFE(svc_model, n_features_to_select=n)
    log_fit = log_rfe.fit(indep_X, dep_Y)
    log_rfe_feature = log_fit.transform(indep_X)
    rfelist.append(log_rfe_feature)
    
    # Get selected columns (features)
    selected_columns = indep_X.columns[log_rfe.support_]
    
    return rfelist, selected_columns

def split_scalar(indep_X, dep_Y):
    X_train, X_test, y_train, y_test = train_test_split(indep_X, dep_Y, test_size=0.25, random_state=0)
    
    # Feature Scaling
    sc = StandardScaler()
    X_train = sc.fit_transform(X_train)
    X_test = sc.transform(X_test)
    
    return X_train, X_test, y_train, y_test, sc

def cm_prediction(classifier, X_test, y_test):
    y_pred = classifier.predict(X_test)
    
    # Making the Confusion Matrix
    cm = confusion_matrix(y_test, y_pred)
    
    Accuracy = accuracy_score(y_test, y_pred)
    report = classification_report(y_test, y_pred)
    return classifier, Accuracy, report, X_test, y_test, cm

def svm_linear(X_train, y_train, X_test, y_test):
    classifier = SVC(kernel='linear', random_state=0)
    classifier.fit(X_train, y_train)
    return cm_prediction(classifier, X_test, y_test)

# Load dataset
dataset1 = pd.read_csv("CKD.csv", index_col=None)
df2 = pd.get_dummies(dataset1, drop_first=True, dtype=int)

indep_X = df2.drop('classification_yes', axis=1)
dep_Y = df2['classification_yes']

# Apply RFE feature selection
rfelist, selected_columns = rfeFeature(indep_X, dep_Y, 6)

# Initialize accuracy list
accsvml = []

# Loop through RFE selected features
for i in rfelist:   
    X_train, X_test, y_train, y_test, sc = split_scalar(i, dep_Y)
    
    classifier, Accuracy, report, X_test, y_test, cm = svm_linear(X_train, y_train, X_test, y_test)  
    accsvml.append(Accuracy)
    
# Compile results
result = pd.DataFrame(accsvml, index=['SVC'], columns=['SVMl'])
print("Selected Columns:", selected_columns)
print(result)

Selected Columns: Index(['al', 'sg_c', 'sg_d', 'rbc_normal', 'dm_yes', 'appet_yes'], dtype='object')
     SVMl
SVC  0.99


In [2]:
import pickle

In [3]:
filename="finalized_model_RFE_classification.sav"

In [4]:
filename1="finalized_model_sc_RFE_classification.sav"

In [5]:
pickle.dump(classifier,open(filename,'wb'))

In [6]:
pickle.dump(sc,open(filename1,'wb'))

In [7]:
loaded_model=pickle.load(open("finalized_model_RFE_classification.sav",'rb'))

In [8]:
loaded_model2=pickle.load(open("finalized_model_sc_RFE_classification.sav",'rb'))

In [9]:
preinput=sc.transform([[3,192,51,0.9,12.6,54]])
preinput

array([[1.75526219e+00, 4.67588763e+02, 1.33083372e+02, 9.03696114e-02,
        2.57247028e+01, 1.31242007e+02]])

In [10]:
result=loaded_model.predict(preinput)

In [11]:
result

array([1])